## Setup 

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
## Imports and environment variables 
import os
os.environ["TOKENIZERS_PARALLELISM"]  = "true"  # set to false if not working

# Core imports 
import torch, numpy as np, pandas as pd, gc,sys, logging, warnings
from torch.utils.data import DataLoader, RandomSampler
from torch.distributions import Categorical
from datasets import load_dataset, load_metric, load_from_disk, DatasetDict
from transformers import (AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, SchedulerType, get_scheduler)
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pytorch_cos_sim
from collections import defaultdict
from accelerate import Accelerator, notebook_launcher
from cachetools import cached, LRUCache
from types import MethodType
from timeit import default_timer as timer
import utils; from utils import *   # local script 
from tqdm.auto import tqdm
import itertools
import copy 
import wandb
from pprint import pprint
from undecorated import undecorated


# Dev imports (not needed for final script)
import seaborn as sns
from IPython.display import Markdown
from pprint import pprint
from IPython.core.debugger import set_trace
from GPUtil import showUtilization
import torchsnooper

In [3]:
logging.basicConfig(format='%(message)s') 
logger = logging.getLogger("main_logger")
logger.setLevel(logging.INFO)

In [4]:
# options for the pp_model 
# 1. tuner007/pegasus_paraphrase
# 2. tdopierre/ProtAugment-ParaphraseGenerator
# 3. eugenesiow/bart-paraphrase

## PEGASUS model
pp_name = "tuner007/pegasus_paraphrase"
pp_tokenizer_pegasus = AutoTokenizer.from_pretrained(pp_name)
pp_model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(pp_name, local_files_only=True)
generate_with_grad = undecorated(pp_model_pegasus.generate)
pp_model_pegasus.generate_with_grad = MethodType(generate_with_grad, pp_model_pegasus)

## BART model
pp_name = "eugenesiow/bart-paraphrase"
pp_tokenizer_bart = AutoTokenizer.from_pretrained(pp_name)
pp_model_bart = AutoModelForSeq2SeqLM.from_pretrained(pp_name, local_files_only=True)
generate_with_grad = undecorated(pp_model_bart.generate)
pp_model_bart.generate_with_grad = MethodType(generate_with_grad, pp_model_bart)


## Functions 

In [8]:
def get_pp_logp(translated): 
    """log(p(pp|orig)) basically.
    works for greedy search, will need tweaking for other types probably"""
    seq_without_first_tkn = translated.sequences[:, 1:]
    attention_mask = pp_model._prepare_attention_mask_for_generation(
        seq_without_first_tkn, pp_tokenizer.pad_token_id, pp_tokenizer.eos_token_id
    )
    scores_log_softmax = torch.stack(translated.scores, 1).log_softmax(2)
    seq_token_log_probs = torch.gather(scores_log_softmax,2,seq_without_first_tkn[:,:,None]).squeeze(-1)
    del scores_log_softmax
    # account for nan values by setting them to 0 (maybe a bit of a hack)
    # will also handle inf and -inf values too by default
    seq_token_log_probs = torch.nan_to_num(seq_token_log_probs)
    # account for the padding tokens at the end 
    seq_token_log_probs = seq_token_log_probs * attention_mask
    seq_log_prob = seq_token_log_probs.sum(-1)
#     if np.any(np.isnan(seq_log_prob.detach().cpu()).tolist()): 
#         warnings.warn(f"Warning: NAN's detected in pp_logp calclulations.\n seq_token_log_probs: {seq_token_log_probs}")
    return seq_log_prob

def get_tokens_from_token_ids_batch(tokenizer, ids_batch):
    l = []
    for i in range(ids_batch.shape[0]): 
        l.append(tokenizer.convert_ids_to_tokens(ids_batch[i,:]))
    return l

def get_start_end_special_token_ids(tokenizer): 
    """The token id's that input/output sequences should start and end with"""
    d = {}
    if pp_tokenizer.name_or_path in ['eugenesiow/bart-paraphrase', 'tdopierre/ProtAugment-ParaphraseGenerator']: 
        d["input_start_id"] =  tokenizer.bos_token_id
        d["input_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
        d["output_start_id"] =  tokenizer.eos_token_id 
        d["output_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
    elif pp_tokenizer.name_or_path == "tuner007/pegasus_paraphrase":
        d["input_start_id"] =  None
        d["input_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id] 
        d["output_start_id"] =  tokenizer.pad_token_id
        d["output_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
    else: 
        raise Exception("unrecognised tokenizer")
    return d

def check_no_nans_or_infs(x):
    assert torch.all(~torch.isnan(x))
    assert torch.all(~torch.isneginf(x))
    assert torch.all(~torch.isposinf(x))

def assert_start_and_end_tokens_are_correct(tokenizer, orig_token_ids, pp_token_ids):
    """Make sure input sequences (orig) and output sequences (pp) start and end with the 
    right special tokens (depends on tokenizer)"""
    start_end_token_d = get_start_end_special_token_ids(pp_tokenizer)
    
    # Input
    if start_end_token_d['input_start_id'] is not None: 
        assert torch.all(orig_token_ids[:,0] == start_end_token_d['input_start_id'])
    # can probs rewrite this to make it nicer but it's fine for now
    assert torch.all(torch.logical_or(orig_token_ids[:,-1] == start_end_token_d['input_end_id'][0], 
                                      orig_token_ids[:,-1] == start_end_token_d['input_end_id'][1]))
    
    # Output
    assert torch.all(pp_token_ids[:,0] == start_end_token_d['output_start_id'])
    assert torch.all(torch.logical_or(pp_token_ids[:,-1] == start_end_token_d['output_end_id'][0], 
                                      pp_token_ids[:,-1] == start_end_token_d['output_end_id'][1]))

def check_scores_for_posinf_nan_and_unexpected_neginf(scores_stacked): 
    """Check we don't have any postive inf or nan, and that all negative inf values are expected"""
    assert torch.all(~torch.isposinf(scores_stacked))
    assert torch.all(~torch.isnan(scores_stacked))

    # We expect to see negative inf for the eos_token when we have not reached min_length. 
    # But we shouldn't expect it for any other tokens
    idx_neginf = torch.nonzero(torch.isneginf(scores_stacked))
    assert torch.all(idx_neginf[:,2] == pp_tokenizer.eos_token_id)
    # Rough check that all idx before min_length are -inf for all elements in batch
    # We do min_length - 1 because sequences are allowed to have length min_length so that idx 
    # shouldn't be set to -inf
    # Not a 100% test but very likely to identify
    assert idx_neginf.shape[0] == (pp_model_params["min_length"] -1) * batch_size  
    # Check that no elements after min_length are -inf
    assert torch.all(idx_neginf[:,1] < (pp_model_params["min_length"] -1 ))

def check_scores_log_softmax_sums_and_shape(scores_log_softmax):
    sums = scores_log_softmax.exp().sum(2)
    # check that the axes is right
    # we want to sum over token probabilities at each generation step, so we 
    # should end up with a shape [batch_size, generated_length]
    assert sums.shape[0] == batch_size  
    assert sums.shape[1] == generated_length - 1
    # check that they sum to 1 along the generated_length axis
    assert torch.allclose(sums, torch.ones(sums.size()), atol = 1e-4)
    
def check_seq_token_log_prob_values_are_correct(): 
    """Just enumerates and checks values
    Quite slow for large batches so run as a test rather than an assert in every batch. 
    """
    l = []
    for i_ex in range(batch_size):
        for i_step in range(generated_length - 1):
            i_tkn = seq_without_first_tkn[i_ex][i_step].item()
            l.append(scores_log_softmax[i_ex,i_step, i_tkn] == seq_token_log_probs[i_ex,i_step])
    assert all(l)
    
def pretty_print_pp_batch_and_next_token_probabilities(): 
    """Goes through each paraphrase and shows at each timestep the next likely tokens. 
    Only will work for greedy search. 
    e.g. [
    "<pad> ['▁My, 0.289', '▁I, 0.261', '▁Hello, 0.07'] | Entropy: 4.23 ",
     "<pad> My ['▁name, 0.935', '▁Name, 0.005', 'name, 0.002'] | Entropy: 0.80 "
    ]
    """
    str_d = defaultdict(list)
    for i_tkn in range(0, generated_length-1): 
        ids = pp_output.sequences[:, :(i_tkn+1)]
        partial_pp = pp_tokenizer.batch_decode(ids)
        kth_ids,kth_probs = tkn_kmaxidx[:, i_tkn, :], tkn_kmaxprob[:, i_tkn, :]
        kth_tkns = get_tokens_from_token_ids_batch(pp_tokenizer, kth_ids)

        # enumerates examples in batch
        z = zip(partial_pp, kth_tkns, kth_probs, ent.detach())
        for i_ex, (ex_sen, ex_next_tkns, ex_next_probs, ex_e) in enumerate(z): 
            # Form nice formatted string mixing together tokens and probabilities
            tkn_tuples_l = [(tkn, round_t(prob,3)) for tkn, prob in zip(ex_next_tkns, ex_next_probs)]
            tkn_str = ['%s, %s' % t for t in tkn_tuples_l]
            # Add to dict of lists and add on entropy term. 
            str_d[i_ex].append(f"{ex_sen} {tkn_str} | Entropy: {ex_e[i_tkn]:.2f} ")

    for v in str_d.values():  pprint(v)

## Exploring paraphrase generation

In [6]:
# INPUT AND PARAMETERS
orig_l = [
    "Look! A small dog. Isn't it cute?", 
    "Far out, if I have to write another sentence...it'll be bad."
]
pp_model_params = {
    "num_beams": 1, 
    "num_return_sequences": 1, 
    "num_beam_groups": 1, 
    "diversity_penalty": 0.,   # must be a float
    "temperature": 1.5,
    "length_penalty" : 1,
    "min_length" : 5
}

## Select which model/tokenizer to use
pp_tokenizer = pp_tokenizer_pegasus
pp_model = pp_model_pegasus

In [10]:
#### TOKENIZER INFORMATION #####
logger.info("\n############################## TOKENIZER ########################################\n")
logger.info(f"We are using the {pp_tokenizer.name_or_path} tokenizer")
logger.info(f"Tokenizer has these special tokens:{pp_tokenizer.all_special_tokens}")
logger.info(f"The bos token is {pp_tokenizer.bos_token} and has id {pp_tokenizer.bos_token_id}")
logger.info(f"The eos token is {pp_tokenizer.eos_token} and has id {pp_tokenizer.eos_token_id}")
logger.info(f"The pad token is {pp_tokenizer.pad_token} and has id {pp_tokenizer.pad_token_id}")
logger.info(f"The unk token is {pp_tokenizer.unk_token} and has id {pp_tokenizer.unk_token_id}")


#### INPUT #####
batch_size = len(orig_l)
orig_tokens = pp_tokenizer(orig_l, return_tensors='pt', padding=True, pad_to_multiple_of=4)
input_length = orig_tokens['input_ids'].size()[1]
orig_l_tokens_list = get_tokens_from_token_ids_batch(pp_tokenizer, orig_tokens['input_ids'])

logger.info("\n############################### INPUT #######################################\n")
logger.info(f"Original text: {orig_l}")
logger.info(f"Batch size is: {batch_size}")
logger.info(f"This is tokenised to get a dict with keys {orig_tokens.keys()} which should be input_ids and attention_mask ")
logger.info(f"The input_ids look like this: {orig_tokens['input_ids']}")
logger.info(f"The tokens are: {orig_l_tokens_list}")
logger.info(f"This has shape {orig_tokens['input_ids'].shape} or [batch_size, input_length], which also\
 might be padded to hit a padding multiple (so input_length is not just the longest example length in the batch).")
logger.info(f"Input length is: {input_length}")
logger.info(f"The attention_mask looks like this: {orig_tokens['attention_mask']}")
logger.info(f"This has shape {orig_tokens['attention_mask'].shape} or [batch_size, input_length]")

##### PARAPHRASE #####
pp_output = pp_model.generate_with_grad(**orig_tokens, **pp_model_params, do_sample=False, 
                                      return_dict_in_generate=True,
                                      output_scores=True,
                                    remove_invalid_values=False)
generated_length = pp_output.sequences.shape[1]
pp_l             = pp_tokenizer.batch_decode(pp_output.sequences, skip_special_tokens=True)
pp_l_with_tokens = pp_tokenizer.batch_decode(pp_output.sequences, skip_special_tokens=False)
pp_l_tokens_list = get_tokens_from_token_ids_batch(pp_tokenizer, pp_output.sequences)

assert_start_and_end_tokens_are_correct(pp_tokenizer, orig_token_ids=orig_tokens['input_ids'],
                                        pp_token_ids=pp_output.sequences)

logger.info("\n#################################### PARAPHRASES ##################################\n")
logger.info(f"Paraphrases: {pp_l}")
logger.info(f"Output has keys {pp_output.keys()}")
logger.info(f"Paraphrases with special tokens: {pp_l_with_tokens}")
logger.info(f"List of pp tokens:{pp_l_tokens_list}")
logger.info(f"Paraphrase token sequences: {pp_output.sequences}")
logger.info(f"Shape of pp token sequences:{pp_output.sequences.shape} or [batch_size, generated_length]")
logger.info(f"Generated length: {generated_length}")

###### SCORES AND PROBABILITIES ########
scores_stacked = torch.stack(pp_output.scores, 1)
# The second argument to stack (i.e. dim) determines which axis the tensors are stacked along. 
# It determines the axis that becomes generated_length - 1
# dim=0 gives shape [generated_length-1, batch_size, vocab_size]
# dim=1 gives shape [batch_size, generated_length-1, vocab_size]
# dim=2 gives shape [batch_size, vocab_size, generated_length-1]
# Our scores_stacked is stacked on dim 1 so it should be second 
assert scores_stacked.shape == torch.Size([batch_size, (generated_length - 1), pp_tokenizer.vocab_size])
check_scores_for_posinf_nan_and_unexpected_neginf(scores_stacked)


# These scores are logits 
# see some of the docs on this page https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/output#transformers.modeling_outputs.Seq2SeqModelOutput
# so we got to take softmax over them 
# but if we take regular softmax then we run into numerical errors
# so instead we take log_softmax
scores_log_softmax = torch.log_softmax(scores_stacked, 2)
check_scores_log_softmax_sums_and_shape(scores_log_softmax)


####### SEQUENCE PROBABILITIES #######
# We select the token probability corresponding to each token 
# However because the scores represent transitions we need to remove the first token from each 
# sequence to match them up. 
seq_without_first_tkn = pp_output.sequences[:,1:]
assert seq_without_first_tkn.shape == torch.Size([batch_size, generated_length - 1])
# Now select prob corresponding to each token
seq_token_log_probs = torch.gather(scores_log_softmax,2,seq_without_first_tkn[:,:,None]).squeeze(-1)
assert seq_token_log_probs.shape == seq_without_first_tkn.shape  # probs should be 1-1 with the filtered tkns
check_no_nans_or_infs(seq_token_log_probs)
# Check that the last token probability corresponds to a possible end token 
output_end_ids = get_start_end_special_token_ids(pp_tokenizer)['output_end_id']
assert all([o in scores_log_softmax[:, -1, output_end_ids] for o in seq_token_log_probs[:,-1]])
check_seq_token_log_prob_values_are_correct()

# The attention mask has 1 everywhere except for where padding tokens occur, where it has 0. 
# It is used to filter out padding tokens from the sequence probablity because then the sequence 
# probability will depend on how many padding tokens there are and the probability of generating them, 
# which (a) we don't want and (b) the probability isn't correct anyway 
attention_mask = pp_model._prepare_attention_mask_for_generation(
    seq_without_first_tkn, pp_tokenizer.pad_token_id, pp_tokenizer.eos_token_id
)
seq_token_log_probs = seq_token_log_probs * attention_mask
# check attention mask only has 0 for padding tokens and not eos tokens or anything else
assert all(seq_without_first_tkn[attention_mask == 0] == pp_tokenizer.pad_token_id)
assert seq_token_log_probs.shape == attention_mask.shape == seq_token_log_probs.shape
assert torch.all(seq_token_log_probs  > -10)  # we shouldn't be picking extrememly rare tokens
seq_log_prob = seq_token_log_probs.sum(-1)
assert seq_log_prob.shape == torch.Size([batch_size])
check_no_nans_or_infs(seq_log_prob)

logger.info("\n##########################  SCORES AND PROBABILITIES ####################################\n")
logger.info(f"Scores is a tuple of length {len(pp_output.scores)} which is one less than the generated_length, or \
the number of tokens in the pp token sequences (this has shape {pp_output.sequences.shape}")
logger.info(f"Each score is a tensor of shape {pp_output.scores[0].shape} or [batch_size, vocab_size]")
#logger.info(f"Full shape:{[o.shape for o in pp_output.scores]}")
logger.info(f"We stack them to get a tensor of shape {scores_stacked.shape} or [batch_size, generated_length - 1, vocab_size]")
logger.info(f"Scores are really logits so we have to take softmax to get probabilities. ")
logger.info("But if we take regular softmax then we run into numerical errors so we take log softmax")
logger.info("We then select the token probability corresponding to each token and sum them to get the log \
probability of the sequence.")

############# ENTROPY AND TOKEN PROBABILITIES ####################
ent = Categorical(logits = scores_stacked).entropy()
assert ent.shape == torch.Size([batch_size, generated_length - 1])
scores_softmax = scores_log_softmax.exp()
k=3
tkn_kmaxprob, tkn_kmaxidx = torch.topk(scores_softmax, k=k, dim=2)
tkn_kmaxprob = tkn_kmaxprob.detach()  # log these 
# The third dimension indexes top1, top2, top3 etc 
assert tkn_kmaxprob[:,:,0].shape == torch.Size([batch_size, generated_length - 1])
# I'd naively expect True everywhere for tkn_kmaxidx[:,:,0] == pp_output.sequences[:, 1:] but it turns 
# out this is not the case because padding tokens seem to have prob 0 and eos tokens are outputted 
# instead by the token generation process and then later replaced by pad

#Uncomment to show how paraphrase is formed. 


logger.info("\n########################## ENTROPY AND TOKEN PROBABILITIES ####################################\n")
logger.info(f"Originals: {orig_l}")
pretty_print_pp_batch_and_next_token_probabilities()


############################## TOKENIZER ########################################

We are using the tuner007/pegasus_paraphrase tokenizer
Tokenizer has these special tokens:['</s>', '<unk>', '<pad>', '<mask_2>', '<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<unk_60>', '<unk_61>', '<unk_62>', '<unk_63>', '<unk_64>', '<unk_65>', '<unk_66>', '<u

["<pad> ['▁Isn, 0.245', '▁A, 0.176', '▁There, 0.091'] | Entropy: 4.71 ",
 '<pad> Isn ["\', 0.881", \'▁the, 0.002\', \'▁it, 0.001\'] | Entropy: 1.66 ',
 "<pad> Isn' ['t, 0.933', 'ta, 0.001', 'T, 0.001'] | Entropy: 0.98 ",
 "<pad> Isn't ['▁it, 0.293', '▁the, 0.29', '▁this, 0.155'] | Entropy: 3.44 ",
 "<pad> Isn't it ['▁cute, 0.577', '▁adorable, 0.109', '▁a, 0.022'] | Entropy: "
 '2.87 ',
 "<pad> Isn't it cute ['▁that, 0.375', '?, 0.357', ',, 0.024'] | Entropy: "
 '3.09 ',
 "<pad> Isn't it cute that ['▁a, 0.405', '▁the, 0.181', '▁there, 0.111'] | "
 'Entropy: 3.32 ',
 "<pad> Isn't it cute that a ['▁small, 0.735', '▁dog, 0.149', '▁little, "
 "0.032'] | Entropy: 1.55 ",
 "<pad> Isn't it cute that a small ['▁dog, 0.911', '▁animal, 0.018', '▁canine, "
 "0.003'] | Entropy: 1.05 ",
 "<pad> Isn't it cute that a small dog ['▁is, 0.729', '?, 0.039', '▁looks, "
 "0.034'] | Entropy: 2.41 ",
 "<pad> Isn't it cute that a small dog is ['▁staring, 0.078', '▁looking, "
 "0.073', '▁there, 0.069'] | Entrop

## Tokenizer differences 

### Types 

Both the "eugenesiow/bart-paraphrase" model and the "tdopierre/ProtAugment-ParaphraseGenerator" are BART tokenizers and have type BartTokenizerFast. The implementation is identical to RobertaTokenizerFast according to the docs, which in turn was derived from GPT-2. They use byte-level Byte Pair Encoding.  

The "tuner007/pegasus_paraphrase" model is a Pegasus tokenizer has type PegasusTokenizerFast. This uses Unigram. 

### Tokenization differences

#### Spaces 

The BART tokenizer has been trained to treat spaces like parts of the tokens (a bit like sentencepiece) so a word will be encoded differently whether it is at the beginning of the sentence (without space) or not:

In [ ]:
tokens = pp_tokenizer_bart(['hello there',' hello there'], return_tensors='pt')
get_tokens_from_token_ids_batch(pp_tokenizer_bart, tokens['input_ids'])

The Pegasus tokenizer doesn't do this

In [ ]:
tokens = pp_tokenizer_pegasus(['hello there',' hello there'], return_tensors='pt')
get_tokens_from_token_ids_batch(pp_tokenizer_pegasus, tokens['input_ids'])

### Representing tokens 

The tokenizers represent tokens differently.  
The BART models use Ġ to indicate start of a word for a token. Its generated tokens look like `['<s>', 'Hello', 'Ġmy', 'Ġname', 'Ġis', 'Ġz', 'f', 'ld', 'lf', 'o', 'q', 'd', '</s>', '<pad>', '<pad>', '<pad>']`  
The Pegasus model uses \_ to indicate start of a word for a token. Its generated tokens look like `['▁Hello', '▁my', '▁name', '▁is', '▁z', 'fl', 'dl', 'fo', 'q', 'd', '</s>', '<pad>']`

### Special tokens

#### BART

In [ ]:
logger.info(f"Tokenizer has these special tokens:{pp_tokenizer_bart.all_special_tokens}")
logger.info(f"The bos token is {pp_tokenizer_bart.bos_token} and has id {pp_tokenizer_bart.bos_token_id}")
logger.info(f"The eos token is {pp_tokenizer_bart.eos_token} and has id {pp_tokenizer_bart.eos_token_id}")
logger.info(f"The pad token is {pp_tokenizer_bart.pad_token} and has id {pp_tokenizer_bart.pad_token_id}")
logger.info(f"The unk token is {pp_tokenizer_bart.unk_token} and has id {pp_tokenizer_bart.unk_token_id}")

#### PEGASUS

In [ ]:
logger.info(f"Tokenizer has these special tokens:{pp_tokenizer_pegasus.all_special_tokens}")
logger.info(f"The bos token is {pp_tokenizer_pegasus.bos_token} and has id {pp_tokenizer_pegasus.bos_token_id}")
logger.info(f"The eos token is {pp_tokenizer_pegasus.eos_token} and has id {pp_tokenizer_pegasus.eos_token_id}")
logger.info(f"The pad token is {pp_tokenizer_pegasus.pad_token} and has id {pp_tokenizer_pegasus.pad_token_id}")
logger.info(f"The unk token is {pp_tokenizer_pegasus.unk_token} and has id {pp_tokenizer_pegasus.unk_token_id}")

### Special token usage with input and output sequences 

#### BART 

They use this format 
```
single sequence: <s> X </s>
pair of sequences: <s> A </s></s> B </s>
```

#### PEGASUS

Format: 
```
- single sequence: ``X </s>``
- pair of sequences: ``A B </s>`` (not intended use)
```

BOS token is never used 

#### Differences 

 Tokenizers also use different tokens when representing input sequences and generating output sentences. Here is a quick summary:

In [ ]:
tokens_d = {
    "bart": {
        "special_tokens": pp_tokenizer_bart.all_special_tokens, 
        "input_start": pp_tokenizer_bart.bos_token,
        "input_end": [pp_tokenizer_bart.pad_token, pp_tokenizer_bart.eos_token], 
        "output_start": pp_tokenizer_bart.eos_token, 
        "output_end": [pp_tokenizer_bart.pad_token, pp_tokenizer_bart.eos_token]
    }, 
    "pegasus": {
        "special_tokens": pp_tokenizer_pegasus.all_special_tokens,
        "input_start": None,
        "input_end": [pp_tokenizer_pegasus.pad_token, pp_tokenizer_pegasus.eos_token], 
        "output_start": pp_tokenizer_pegasus.pad_token,  
        "output_end": [pp_tokenizer_pegasus.pad_token, pp_tokenizer_pegasus.eos_token], 
    }
}
tokens_d

### Token indexing

In [ ]:
def print_tokens_from_ids(tokenizer, start_id=100, end_id=200):
    ids = list(range(start_id,end_id))
    print(*list(zip(ids, tokenizer.convert_ids_to_tokens(ids))))

#### BART 

Having a look at generated tokens makes me suspect that they are indexed in whatever order they are encountered in the source text they are trained on. It seems like a rough frequency of english tokens but there are also tokens that are definitely out of order. 

The first few are reserved for special tokens, and the other low numbers (e.g. up to 100) are pretty common suffixes and words

In [ ]:
print_tokens_from_ids(pp_tokenizer_bart, 0,50)

Looking at 100 to 200 you can see some words (e.g. Trump at 140, or 2017 at 193) that aren't common enough to be that high. This makes me suspect that words are in encounter order in the text. 

In [ ]:
print_tokens_from_ids(pp_tokenizer_bart, 100,200)

Tokens towards the end are gibberish or mispellings encountered in the input. The fifth last token is something labelled <|endoftext|> and I don't know what that is. Then there is a bunch of tokens like "madeupword0001". The last token is the mask token and then token indicies after that return None. 

In [ ]:
print_tokens_from_ids(pp_tokenizer_bart, pp_tokenizer_bart.vocab_size-20, pp_tokenizer_bart.vocab_size+10)

#### PEGASUS

Special tokens make up the first hundred or so. After that there's a token \<n> that seems like some new line thing. 

In [ ]:
print_tokens_from_ids(pp_tokenizer_pegasus, 0,120)

Unlike the BART models I can believe that these tokens are in order of frequency. I can't see anything that is obviously out of place. 

In [ ]:
print_tokens_from_ids(pp_tokenizer_pegasus, 120,250)

There's nothing special at the end, just looks like isolated tokens and None values after the tokens finish. It's also worth noting that the Pegasus model has ~96100 tokens which is way more than the ~50270 of the BART models (almost double). 

In [ ]:
print_tokens_from_ids(pp_tokenizer_pegasus, pp_tokenizer_pegasus.vocab_size-20, pp_tokenizer_pegasus.vocab_size+10)

## Questions 

### When does the model generate padding tokens?

For both models padding tokens are generated after the EOS token. Additionally for Pegasus generated text starts with the padding token. 

### Why do generated paraphrases start with the EOS token? 

This is only the case for BART models. For pegasus models they use a padding token to start generated paraphrases. 

I don't know what the BOS token isn't used for these things. Pegasus has an open issue [here](https://github.com/huggingface/transformers/issues/12474). 

Whatever the reason you should just do the default because that is what the preprocessing does and you will get the best results that way. 

### Does p(PAD) =1 after an eos token?

For both BART and Pegasus models it appears that probability of outputting a pad token is actually zero at all timesteps. Instead the model outputs the eos token over and over, and there must be some post-processing that takes place that replaces eos token with padding token. 
For Pegasus it appears it is the same behaviour. 
Example code: 

In [ ]:
print(round_t(scores_softmax[:,:,pp_tokenizer.eos_token_id]))
print(round_t(scores_softmax[:,:,pp_tokenizer.pad_token_id]))

What is interesting is that there is probability assigned to other tokens other than eos and pad after a eos token is outputted. Again there must be some kind of postprocessing that takes care of this situation because I haven't really seen it in the wild. 

Some models (e.g. GPT2) don't even have a PAD token. Instead they use the eos token on repeat. See this [issue](https://github.com/huggingface/transformers/issues/8452#issuecomment-739008168). What is confusing is seeing this behaviour with models that have a padding token. 

### Do rows always sum to 1 when looking at token generation scores? 


Yes, they should. I put in an assert to check this. 
If you have a nan or an inf then they won't sum to 1. To confirm this

In [ ]:
print(torch.isnan(torch.sum(torch.tensor([1,2,3, torch.nan]))))
print(torch.isinf(torch.sum(torch.tensor([1,2,3, torch.inf]))))

### Does first row sum to 0? (the one corresponding to the startoff token)


So there is no token scores that correspond to the first token (usually a bos or pad token). The scores are a tuple of length (`generated_length - 1`). So there shouldn't be a "zero" row really. 
I remember seeing something like this at some point so I'll keep an eye out for it. 

### How are logits containing nan or inf transformed with softmax and log_softmax?

We can explore this through some code examples.

#### Vanilla case  
First we look at the case without any nan or inf. 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

The softmax values are interpreted as probabilities, and the log softmax is just the log of the probabilities, done for numerical stability. We can just take exponents to return to probabilities if needed. 

In [ ]:
print(torch.log_softmax(logits,0).exp())

#### Positive inf

Now let's see what happens if we introduce a positive inf. 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2, torch.inf])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

We get nan values in the softmax and log_softmax.  So if you see nans in the softmax, remember that an inf in the scores is one reason why it may happen.  

This is interesting because if we just assume inf is a large positive number, we'd expect a softmax with basically a 1 and all zeros, and a log softmax of a 0 and a lot of negatives. We can try it here: 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2, 10000000000])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

Basically what we get. So this indicates that if we get a positive inf we might be able to mitigate this problem by clipping it to some kind of maximum value. 

#### Negative inf 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2, -torch.inf])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

Negative inf behaves a bit differently. The softmax is unaffected and basically just assigns a prob of 0 to the corresponding entry. The log softmax carries the `-inf` through. 

Again clipping the -inf to a large negative value can mitigate this problem somewhat: 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2, -10000000])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

#### nan values 

In [ ]:
logits = torch.tensor([1.4, -1, 3, 2, torch.nan])
print(logits)
print(torch.softmax(logits,0))
print(torch.log_softmax(logits,0))

A nan in the logits propagates and affects the entire softmax and log_softmax tensors. The network basically gives up and says "no idea how to deal with this. 

This seems to be the case with most torch functions; e.g.  

In [ ]:
print(torch.sum(logits,0))
print(torch.divide(logits,0.2))
print(logits + logits)

### How do you interpret token entropy?

The token scores (when stacked) are a tensor of dimensions (batch_size, generated_length - 1, vocab_size). We take softmax to get a tensor of probability distributions across all possible tokens. We can also calculate the entropy of each of these probability distributions. 

Entropy is a measure of how "peaky" or "flat" a probability distribution is. It is the expected value of the self-information of an event, which is basically a measure of how "surprised" you would be if that event occured. 
If we have a discrete random variable $X$ with probability distribution $P(x) $, entropy is given by $$H(X) = \mathbb{E}_{ X\sim P} [I(x)] = -\mathbb{E}_{X \sim P} [\log P(x)] $$ which is practically calculated by $$H(X) =  -\sum_{x=-\infty}^\infty p(x) \log(p(x))$$

The lowest value of entropy is 0, which is when you have $P(x)=1$ for some event. 

In [ ]:
Categorical(probs = torch.tensor([1,0,0,0])).entropy()

High values of entropy occur when the probability distribution is very flat. 

In [ ]:
print(Categorical(probs = torch.tensor([0.20,0.50,0.10,0.20])).entropy())  # spikier, lower entropy
print(Categorical(probs = torch.tensor([0.25,0.25,0.25,0.25])).entropy())  # flatter, higher entropy

In terms of tokens, we can show two realistic distributions below. The first has two likely tokens, one somewhat likely, and the rest unlikely. The second has many more likely tokens. 

In [ ]:
l1 = [0.001,0.001,0.001,0.001,0.5,0.001,0.4,0.001,0.001,0.001,0.001,0.001,0.09]
l2 = [0.1,0.1,0.1,0.1,0.025,0.1,0.1,0.1,0.1,0.1,0.025,0.025,0.025]
print(Categorical(probs = torch.tensor(l1)).entropy())  # spikier, lower entropy
print(Categorical(probs = torch.tensor(l2)).entropy())  # flatter, higher entropy

There isn't a theoretical maximum for entropy, but for tokens you'll be governed by vocab size. Here we show some practical maximums for some different vocab sizes

In [ ]:
v_size = 1000
print(Categorical(probs = torch.tensor([1/v_size for i in range(v_size)])).entropy())  # v small vocab
v_size = 10000
print(Categorical(probs = torch.tensor([1/v_size for i in range(v_size)])).entropy())  # small vocab
v_size = 50000
print(Categorical(probs = torch.tensor([1/v_size for i in range(v_size)])).entropy())  # ~BART vocab
v_size = 100000
print(Categorical(probs = torch.tensor([1/v_size for i in range(v_size)])).entropy())  # ~PEGASUS vocab

These can be a bit hard to interpret so maybe you should also look at some other token-level stats, like max_prob, second_max_prob, third_max_prob, mean, variance or other things like that. 

Some other things to note. First you still get an entropy value if your probability dist sums up to more than 1, so make sure to check this before doing entropy. Secondly if you have nan or inf in the probability values then you will get an error. This is true if you use either `probs` or `logits` in the Categorical function. 

In [ ]:
print(Categorical(probs = torch.tensor([0.5,0.25,0.25,0.25])).entropy())       # sums to more than 1, gives result
#print(Categorical(probs = torch.tensor([0.5,0.25,0.25,torch.nan])).entropy())  # throws error
#print(Categorical(probs = torch.tensor([0.5,0.25,0.25,torch.inf])).entropy())  # throws error
#print(Categorical(logits = torch.tensor([0.5,0.25,0.25,torch.nan])).entropy())  # throws error

### What does it mean to take average of token entropy? 

Average entropy over tokens will depend on how many padding tokens are added. This in turn depends on how many examples there are in the batch.   
The entropy of padding tokens appears to be quite high based on some quick experiments.  
It might or might not depend on sentence length.   
It could be useful as a broad measure to see if the model is getting more "peaky" in selecting tokens at each time step.   
It might be useful when tracking a paraphrase over time and seeing its variations?   

### How do you get nan and inf introduced into token scores?


My understanding is that you get -inf for the first (min_length - 1) steps when you introduce a min_length parameter for the generated sequences for the eos_token_id slot. This is to stop the token from appearing and truncating the sequence. 

You might also get -inf when setting other parameters to the `generate()` function (e.g. bad_words_ids or something similar).

Wrote an assert for this. 

Things to check 

* how big is the action space? 
  * initial estimate: on the order of vocab_size ^ sequence length (so bloody huge)
  * but only a very small proportion are valid actions
* how many pp has probs over: 1e-5, 1e-4, 1e-3, 1e-2, 1e-1. would be a good plot. x axis epoch, then ether do (a) for individual examples, or (b) as averages across examples
* log top X sampled sentences and their probs (maybe probs can be a graph)

* how does padding mask affect things?
* how does token-type-ids affect things? 
* which dist do you calculate KL divergence and entropy over? 
  * is it the token entropy’s at each generation step? 
  * is it the entropy of the generated paraphrase tokens? 
* given size of action space is this a good candidate for differential entropy? 
* when do you hit floating point threshold for token probabilities? When do nans and inf get introduced? 
* does using fp32 affect token calculations? 
* how does dropout affect generated probabilities? How does train/eval mode affect generated probs for a sentence? 
* how does layer-norm affect the probs? 

## Generation terms 

### Encoder_input_ids

In [ ]:
### Decoder_input_ids

In [ ]:
### token_type_ids

In [ ]:
### attention_mask

In [ ]:
### input_ids

In [ ]:
### decoder_start_token_id